In [33]:
from core.DQNAgent import DQNAgent
import gym
import numpy as np
import torch
from tqdm import tqdm
import glob
import os
from IPython.display import Video

import pyvirtualdisplay
_display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
_ = _display.start()

def plot_videos(path='.'):
  stringa = 'ffmpeg -i \"concat:'
  elenco_video = glob.glob(f'{path}/*.mp4')
  if len(elenco_video) == 0:
      print('0 mp4 found in this path')
      return
  elenco_file_temp = []
  for f in elenco_video:
    file = 'temp' + str(elenco_video.index(f) + 1) + '.ts'
    os.system('ffmpeg -i ' + f + ' -c copy -bsf:v h264_mp4toannexb -f mpegts ' + file)
    elenco_file_temp.append(file)
  for f in elenco_file_temp:
    stringa += f
    if elenco_file_temp.index(f) != len(elenco_file_temp)-1:
      stringa += '|'
    else:
      stringa += f'\" -c copy  -bsf:a aac_adtstoasc {path}/output.mp4'
  os.system(stringa)
  display(Video(f'{path}/output.mp4', embed=True))

In [22]:
np.random.seed(1000)

env = gym.make("CartPole-v1")
env = gym.wrappers.Monitor(env, 'recording', force=True, video_callable=lambda episode_id: True)
agent = DQNAgent(env)

In [23]:
DISCOUNT_FACTOR = 0.99
LEARNING_RATE = 0.00005
total_rewards = []
number_steps = []
cost_function_means = []

In [24]:
for i in range(10):
    total_reward, steps, mean_cost_function = agent.start_episode_and_evaluate(DISCOUNT_FACTOR, LEARNING_RATE, 0, render=True, optimize=False)
    total_rewards.append(total_reward)
    total_rewards.append(total_reward)
    total_rewards.append(total_reward)
env.close()

In [34]:
plot_videos('recording')